# Sep.4th Meeting 

## One existing bug

If we are doing finite difference in x direction:
if mod(Nx,TILE_DIM_X) = 1: bug even HALO_WIDTH >= 2
if mod(Nx,TIME_DIM_X) >= 2: looks okay


One way to avoid this: set TILE_DIM according to the value of Nx to avoid this

Nx = 5

TILE_DIM = 2, 4 : bug
but TILE_DIM = 3: no bug  mod(5,3) == 2



Nx = Ny = 1001

D2y

TILE_DIM = (4,16)
 
TILE_DIM = (16,4)


## Current status

All functions have passed tester_function_v3() with conditions that would cause bugs in the above

There might be still bugs in Hxinv_GPU_shared and Hyinv_GPU_shared because the GPU results and CPU results in the myMAT!() function are off by a fixed ratio for each elements.

There might be still bugs in Hx_GPU_shared() used in the last step. I think I fixed the bug in Hy_GPU_shared().



One bug in previous implementation of Conjugate_Gradient()

I define 
```
mutable struct intermediates_GPU_mutable
    Nx::Int64
    Ny::Int64
    N::Int64
    # du_ops::CuArray{Float64,1}
    du_x::CuArray{Float64,1}
    du_y::CuArray{Float64,1}
    du_ops::CuArray{Float64,1}
    du1::CuArray{Float64,1}
    du2::CuArray{Float64,1}
    du3::CuArray{Float64,1}
    du4::CuArray{Float64,1}
    du5::CuArray{Float64,1}
    du6::CuArray{Float64,1}
    du7::CuArray{Float64,1}
    du8::CuArray{Float64,1}
    du9::CuArray{Float64,1}
    du10::CuArray{Float64,1}
    du11::CuArray{Float64,1}
    du12::CuArray{Float64,1}
    du13::CuArray{Float64,1}
    du14::CuArray{Float64,1}
    du15::CuArray{Float64,1}
    du16::CuArray{Float64,1}
    du17::CuArray{Float64,1}
    du0::CuArray{Float64,1}
    du::CuArray{Float64,1}
end
```

and I define
```
cu_zeros = CuArray(zeros(N))
```

Then I define
```
iGm = intermediates_GPU_mutable(Nx,Ny,N,cu_zeros,cu_zeros, ...)
```

But this is actually create a pointer from cu_zeros to du_x, du_y, ...
After du_x is rewritten, cu_zeros will be altered, and everything else would be messed up.

Instead, I should do
```
iGm = intermediates_GPU_mutable(Nx,Ny,N,CuArray(zeros(N)),CuArray(zeros(N)),...)
```
to prevent such data hygiene issues. It took me a while to realize this after I accidentally checked the value of cu_zeros.


I think i would need to rewrite GPU conjugate gradient code a little bit even if it works.

